# Gün 4, bölüm 2: Salgın tahminleri

<div style="background: #f8f9fa; padding: 0.5rem 1rem; border-radius: 8px; margin-bottom: 1rem; border-left: 4px solid #007bff;">
<a href="../../index.ipynb" style="text-decoration: none; color: #007bff; font-weight: bold;">← Kurs İndeksine Geri Dön</a> | 
<a href="../../index-tr.ipynb" style="text-decoration: none; color: #dc3545; font-weight: bold;">🇹🇷 Türkçe İndeks</a>
</div>

---

## Giriş

Bu uygulama, bir Ebola Virüs Hastalığı (EVD) salgınının erken değerlendirmesini ve yeniden yapılandırılmasını simüle eden uygulamanın ikinci (iki bölümden) bölümüdür. 
**Bölüm 2**'ye başlamadan önce **bölüm 1**'i tamamladığınızdan emin olun. 
Uygulamanın **bölüm 2**'sinde, büyüme oranı tahmini, temas izleme verileri, gecikmeler ve bulaştırıcılık tahminleri dahil olmak üzere salgının erken aşamasının analizinin çeşitli yönlerini tanıtıyoruz. 


## Öğrenme çıktıları

Bu uygulamanın **bölüm 2**'sinin sonunda şunları yapabilmelisiniz:

- Salgının büyüme oranını ve ikiye katlanma süresini tahmin etme ve yorumlama

- Bulaştıran / bulaşan birey çiftlerine dair verilerden seri aralığı tahmin etme

- Salgının üreme sayısını tahmin etme ve yorumlama

- Kısa vadeli gelecekteki insidansı öngörme

## Bağlam: Batı Afrika'da kurgusal bir ülkede yeni bir EVD salgını

Batı Afrika'da kurgusal bir ülkede yeni bir EVD salgını bildirildi. Sağlık Bakanlığı salgın müdahalesini koordine etmekle görevli ve sizi gerçek zamanlı müdahaleyi bilgilendirmek için salgın analizi konusunda danışman olarak görevlendirdi. Verileri okudunuz ve tanımlayıcı analiz yaptınız (**bölüm 1**). Şimdi biraz istatistiksel analiz yapalım! 

## Gerekli paketler 

CRAN veya github'da mevcut olan aşağıdaki paketler bu analiz için gereklidir.
Bunları **bölüm 1**'de kurmuş olmanız gerekir ancak kurmediyseniz, gerekli paketleri şu şekilde kurun:

Paketler kurulduktan sonra yeni bir R oturumu açmanız gerekebilir. Ardından kütüphaneleri şu şekilde yükleyin:

In [ ]:
library(readxl)
library(outbreaks)
library(incidence)
library(epicontacts)
library(distcrete)
library(epitrix)
library(EpiEstim)
library(projections)
library(ggplot2)
library(magrittr)
library(binom)
library(ape)
library(outbreaker2)
library(here)

## [Bölüm 1](./real-time-response-1.html)'de işlenen verileri okuma

<!--
ZNK: These two chunks are needed because of the way the documents are structured
in blogdown. The source that we edit is not the same as the site that is
rendered. Everything in this directory will end up in the same directory as the
"static" when the website is displayed, but the current directory structure is
present when the practicals are built, so we have to do this silly 
song-and-dance to appease the directory gods.
-->

In [ ]:
i_daily <- readRDS(here("data/clean/i_daily.rds"))
i_weekly <- readRDS(here("data/clean/i_weekly.rds"))
linelist <- readRDS(here("data/clean/linelist.rds"))
linelist_clean <- readRDS(here("data/clean/linelist_clean.rds"))
contacts <- readRDS(here("data/clean/contacts.rds"))

## Log-lineer model kullanarak büyüme oranını tahmin etme

İnsidansın en basit modeli muhtemelen log-lineer model, yani log-dönüştürülmüş insidanslarda lineer regresyondur. 
Bunun için sıfır insidansla (log alınamayan) çok fazla sorun yaşamamak için haftalık insidansla çalışacağız. 

Log-dönüştürülmüş insidansı görselleştirin:

In [ ]:
ggplot(as.data.frame(i_weekly)) + 
  geom_point(aes(x = dates, y = log(counts))) + 
  scale_x_incidence(i_weekly) +
  xlab("tarih") +
  ylab("log haftalık insidans") + 
  theme_minimal()

Bu çizim size salgın hakkında ne söylüyor?   

`incidence` paketinde, `fit` fonksiyonu bu modelin parametrelerini bir insidans nesnesinden (burada, `i_weekly`) tahmin edecektir. Bunu verilere uygulayın ve sonucu `f` adında yeni bir nesnede saklayın. `f`'yi yazdırabilir ve büyüme oranı r ve ikiye katlanma süresinin tahminlerini türetmek için kullanabilir ve ilgili modeli insidans çizmine ekleyebilirsiniz:  

Haftalık insidans verilerine log-lineer model uydurma:  

In [ ]:
f <- incidence::fit(i_weekly)
f
plot(i_weekly, fit = f)

Çizime ve uyuma bakarak, bunun makul bir uyum olduğunu düşünüyor musunuz?   

## Gözlenen gecikmelere dayalı olarak log-lineer model için uygun eşik tarihi bulma

Yukarıda çizdiğiniz log(insidans) çizimini kullanarak ve en son haftalarda neden üstel büyüme gözlemlenmeyebileceğini düşünerek, bir eşik tarihi seçin ve log-lineer modeli büyüme oranı r'yi ve ikiye katlanma süresini daha güvenilir bir şekilde tahmin edebileceğimizi düşündüğünüz epieğrinin uygun bir bölümüne uydurma.  

Vakaların semptom başlangıcından ne kadar sonra hastaneye yatırıldığını incelemek isteyebilirsiniz; 
bu, aşağıdaki gibi seçeceğiniz eşik tarihi konusunda bilgi verebilir:  

In [ ]:
summary(as.numeric(linelist_clean$date_of_hospitalisation - linelist_clean$date_of_onset))

In [ ]:
# epieğrinin sonunda kaç hafta atmamız gerekiyor
n_weeks_to_discard <- 

In [ ]:
min_date <- min(i_daily$dates)
max_date <- max(i_daily$dates) - n_weeks_to_discard * 7
# haftalık kesik insidans
i_weekly_trunc <- subset(i_weekly, 
                         from = min_date, 
                         to = max_date) # son birkaç haftalık veriyi at
# günlük kesik insidans (lineer regresyon için kullanılmaz ama daha sonra kullanılabilir)
i_daily_trunc <- subset(i_daily, 
                         from = min_date, 
                         to = max_date) # son iki haftalık veriyi kaldır

Log-lineer modelinizi daha önce olduğu gibi yeniden uydurma ve çizme ancak kesik veri `i_weekly_trunc` kullanma. 

In [ ]:
f <- incidence::fit(i_weekly_trunc)
f
plot(i_weekly_trunc, fit = f)

Uyumunuzun özet istatistiklerine bakın:

In [ ]:
summary(f$model)

Uyum iyiliğini (Rsquared), tahmin edilen eğimi (büyüme oranı)
ve karşılık gelen ikiye katlanma süresini aşağıdaki gibi görebilirsiniz:  

In [ ]:
# model verilere iyi uyuyor mu? 
adjRsq_model_fit <- summary(f$model)$adj.r.squared
# salgının tahmin edilen büyüme oranı nedir?
daily_growth_rate <- f$model$coefficients['dates.x']
# güven aralığı:
daily_growth_rate_CI <- confint(f$model, 'dates.x', level=0.95)
# salgının ikiye katlanma süresi nedir? 
doubling_time_days <- log(2) / daily_growth_rate
# güven aralığı:
doubling_time_days_CI <- log(2) / rev(daily_growth_rate_CI)


Log-lineer erken salgın değerlendirmesi için basit ve hızlı bir yöntem olmasına rağmen, sadece salgın büyümesinin olduğu noktaya kadar uydurma konusunda dikkatli olunmalıdır. Bu noktayı tanımlamanın zor olabileceğini unutmayın.  

## Temas İzleme - Temasları inceleme

Temas izleme bir Ebola salgın müdahalesinin temel taşlarından biridir. Bu, bilinen bir vaka ile temas kurmuş (yani enfekte olmuş olabilir) risk altındaki bireyleri belirlemeyi ve takip etmeyi içerir. Ebola için temaslı kişiler 21 gün boyunca takip edilir (kuluçka döneminin üst sınırı). Bu, semptomatik hale gelen temaslı kişilerin hızla izole edilebilmesini sağlar ve daha fazla bulaşma şansını azaltır. Burada tarihlerde hata olan girişleri attığımız `linelist_clean` yerine tam çizgi listesini kullanıyoruz. Ancak temas yine de geçerli olabilir.

`epicontacts` paketindeki `make_epicontacts` fonksiyonunu kullanarak, `epi_contacts` adında yeni bir `epicontacts` nesnesi oluşturun. İlgili `to` ve `from` argümanlarının sütun isimlerini kontrol ettiğinizden emin olun. 

In [ ]:
epi_contacts <- make_epicontacts(linelist = linelist, contacts = contacts, 
                                 id = "case_id", # çizgi listesinde tanımlayıcının adı
                                 from = "infector", # temaslar içinde 'from' sütununun adı
                                 to = "case_id",  # temaslar içinde 'to' sütununun adı
                                 directed = TRUE)

In [ ]:
epi_contacts

Bu temasları kolayca çizebilirsiniz, ancak biraz düzenleme ile (`?vis_epicontacts`'a bakın) örneğin şekilleri cinsiyete göre ve ok renklerini maruz kalma kaynağına göre (veya ilginizi çeken diğer değişkenlere göre) özelleştirebilirsiniz:

In [ ]:
# örneğin, temasların bildirilen enfeksiyon kaynağına bakın.
table(epi_contacts$contacts$source, useNA = "ifany")


In [ ]:
p <- plot(epi_contacts, node_shape = "gender", shapes = c(m = "male", f = "female"), node_color = "gender", edge_color = "source", selector = FALSE)
p

In [ ]:
p <- plot(epi_contacts, node_shape = "gender", shapes = c(m = "male", f = "female"), node_color = "gender", edge_color = "source", selector = FALSE)
#learn::save_and_use_widget(p, 'real-time-response-2-nework.html')

`match` fonksiyonunu kullanarak (`?match`'a bakın) görselleştirilen temasların gerçekten vaka olduğunu kontrol edin.

In [ ]:
match(contacts$case_id, linelist$case_id)

Bunların gerçekten hepsinin vaka olduğundan memnun kaldıktan sonra ağa bakın:  

- süper yayılım (heterojen bulaşma) var gibi görünüyor mu?
- vakaların cinsiyetine bakarak bundan bir şey çıkarabilir misiniz? Cinsiyete göre görünür farklar var mı?

## Bulaştırıcılığı tahmin etme (`$R$`)  

### Dallanma süreci modeli  

Hastalığın bulaştırıcılığı, enfekte vaka başına beklenen ikincil vaka sayısı olarak tanımlanan üreme sayısı R'nin tahmini yoluyla değerlendirilebilir. Bir salgının erken aşamalarında ve bağışıklığı olmayan büyük bir popülasyon varsayıldığında, bu miktar aynı zamanda temel üreme sayısı $$R_0$$'dır, yani tamamen duyarlı büyük bir popülasyonda $$R$$.  

`EpiEstim` paketi, semptom başlangıcı tarihlerini ve seri aralık dağılımı bilgisini, yani vakadaki semptom başlangıcından bulaştırıcısındaki semptom başlangıcına kadar geçen sürenin dağılımını kullanarak $$R$$'nin Bayesian tahminini uygular (Cori ve ark., 2013, AJE 178: 1505–1512'ye bakın).

Kısaca, `EpiEstim` belirli bir gündeki insidansı Poisson dağılımlı olarak tanımlayan basit bir model kullanır ve ortalaması o gündeki toplam enfeksiyon kuvveti tarafından belirlenir:

$$ I_t  ∼  Poisson(\lambda_t)$$

burada $$I_t$$ $$t$$ günündeki insidans (semptom başlangıcına dayalı) ve $$\lambda_t$$ o gündeki enfeksiyon kuvvetidir. R'yi üreme sayısı ve w()'yi ayrık seri aralık dağılımı olarak belirtirsek:

 $$\lambda_t = R \sum_{s=1}^t I_sw(t-s)$$

Olabilirlik (model ve parametreler verildiğinde verileri gözlemleme olasılığı) R'nin bir fonksiyonu olarak tanımlanır:

 $$L(I) = p(I|R) = \prod_{t = 1}^{T} f_P(I_t, \lambda_t)$$

burada $$f_P(.,\mu)$$ ortalaması $$\mu$$ olan Poisson dağılımının olasılık kütle fonksiyonudur.

### Seri aralığı tahmin etme

Bulaştıran ve bulaşan birey çiftleri hakkında veri toplandığından, bu seri aralık dağılımını tahmin etmek için yeterli olmalıdır. Durum böyle olmasaydı, bunun yerine geçmiş salgınlardan veri kullanabilirdik.  

`get_pairwise` fonksiyonu seri aralığı (yani bulaştıranlar ve bulaşan bireyler arasındaki başlangıç tarihi farkı) çıkarmak için kullanılabilir:

In [ ]:
si_obs <- get_pairwise(epi_contacts, "date_of_onset")
summary(si_obs)
## Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
## 1.000   5.000   6.500   9.117  12.250  30.000 
hist(si_obs, breaks = 0:30,
     xlab = "Semptom başlangıcından sonra günler", ylab = "Frekans",
     main = "Seri aralık (ampirik dağılım)",
     col = "grey", border = "white")


Bu dağılım hakkında ne düşünüyorsunuz? Gerekli gördüğünüz herhangi bir ayarlama yapın ve sonra `epitrix` paketindeki `fit_disc_gamma` fonksiyonunu kullanarak bu verilere ayrıklaştırılmış Gamma dağılımı uydurma. Sonuçlarınız yaklaşık olarak şöyle görünmelidir:  

In [ ]:
si_fit <- fit_disc_gamma(si_obs, w = 1)
si_fit

`si_fit` uydurulmuş gecikmeler hakkında `$distribution` yuvasındaki tahmin edilen dağılım dahil olmak üzere çeşitli bilgiler içerir. Bu dağılımı aşağıdaki çizimde ampirik verilerle karşılaştırabilirsiniz:

In [ ]:
si <- si_fit$distribution
si

## uydurulmuş dağılımı karşılaştır
hist(si_obs, xlab = "Semptom başlangıcından sonra günler", ylab = "Frekans",
     main = "Seri aralık: verilere uyum", col = "salmon", border = "white",
     50, ylim = c(0, 0.15), freq = FALSE, breaks = 0:35)
points(0:60, si$d(0:60), col = "#9933ff", pch = 20)
points(0:60, si$d(0:60), col = "#9933ff", type = "l", lty = 2)


Bu nesil süresi tahminini güvenir miydiniz? Bunu Batı Afrika EVD salgınından gerçek tahminlerle (WHO Ebola Response Team (2014) NEJM 371:1481–1495) ortalama 15.3 gün ve standart sapma 9.3 gün ile nasıl karşılaştırırsınız?  

### Üreme Sayısının Tahmini

Artık seri aralık tahminlerimiz olduğuna göre, bu bilgiyi hastalığın bulaştırıcılığını (`$R_0$` ile ölçülen) tahmin etmek için kullanabiliriz. Üstel büyüme olduğuna karar verdiğiniz döneme kesilmiş günlük (haftalık değil) insidans nesnesini (`i_daily_trunc`) kullandığınızdan emin olun.  

Az önce elde ettiğiniz seri aralığın ortalama ve standart sapma tahminlerini kullanarak, üreme sayısını tahmin etmek için `estimate_R` fonksiyonunu kullanın (`?estimate_R`'ye bakın) ve sonucu `R` adında yeni bir nesnede saklayın.  

`estimate_R` kullanmadan önce, `make_config` fonksiyonunu kullanarak bir `config` nesnesi oluşturmanız gerekir; burada üreme sayısını tahmin etmek istediğiniz zaman penceresini ve kullanılacak `mean_si` ve `std_si`'yi belirteceksiniz. Zaman penceresi için `t_start = 2` kullanın (geçmişte gözlemlenen insidansa koşullandırdığınız için R'yi ancak 2. günden itibaren tahmin edebilirsiniz) ve kesik insidansınızın `i_daily_trunc` son tarihine kadar R tahmin etmek için `t_end = length(i_daily_trunc$counts)` belirtin.  

In [ ]:

config <- make_config(mean_si = si_fit$mu, # daha önce tahmin edilen si dağılımının ortalaması
                      std_si = si_fit$sd,  # daha önce tahmin edilen si dağılımının standart sapması
                      t_start = 2,         # zaman penceresinin başlangıç günü
                      t_end = length(i_daily_trunc$counts)) # zaman penceresinin son günü


In [ ]:
R <- estimate_R(i_daily_trunc, method = "parametric_si", config = config)
plot(R, legend = FALSE)

Üreme sayısı için medyan ve %95 güvenilirlik aralıklarını (%95CrI) şu şekilde çıkarın:

In [ ]:
R_median <- R$R$`Median(R)`
R_CrI <- c(R$R$`Quantile.0.025(R)`, R$R$`Quantile.0.975(R)`)
R_median
R_CrI

Bu sonuçları yorumlayın: üreme sayısı hakkında ne düşünüyorsunuz? Neyi yansıtıyor? Epieğrinin son bölümüne dayanarak, bazı meslektaşlar insidansın düştüğünü ve salgının kontrol altına alınmış olabileceğini öne sürüyor. Bu konudaki görüşünüz nedir?

R0'ı doğrudan büyüme oranından ve seri aralıktan tahmin edebileceğinizi unutmayın, 
Wallinga ve Lipsitch, Proc Biol Sci, 2007'de açıklanan formülü kullanarak: 
$$R_0 = \frac{1}{\int_{s=0}^{+\infty}e^{-rs}w(s)ds}$$ , ve `epitrix` paketinin `r2R0` fonksiyonunda uygulanmıştır. Bu karmaşık bir formül gibi görünse de, arkasındaki mantık basittir ve aşağıdaki şekilde gösterilmiştir: gözlemlenen üssel olarak büyüyen bir insidans eğrisi için, 
seri aralığı biliyorsanız, üreme sayısını türetebilirsiniz. 

![Büyüme oranından ve seri aralıktan R0 tahmin etme.](../static/img/R0fromr.png)

Yukarıdaki şekille karşılaştırıldığında, büyüme oranı r'de belirsizlik vardır ve seri aralık tek bir değer yerine tam bir dağılıma sahiptir. Bu, R'yi tahmin ederken aşağıdaki gibi hesaba katılabilir:

In [ ]:
# yukarıda tahmin ettiğimiz büyüme oranı ve seri aralıktan R0 tahminlerinin bir örneği oluştur 
R_sample_from_growth_rate <- lm2R0_sample(f$model, # büyüme oranı r tahminlerimizi içeren log-lineer model
                                          si$d(1:100), # seri aralık dağılımı (100 gün sonra kesilmiş)
                                          n = 1000) # istenen örnek boyutu
# bunları çiz:
hist(R_sample_from_growth_rate)
# medyan nedir?
R_median_from_growth_rate <- median(R_sample_from_growth_rate)
R_median_from_growth_rate # R_median ile karşılaştır
# %95GA nedir?
R_CI_from_growth_rate <- quantile(R_sample_from_growth_rate, c(0.025, 0.975))
R_CI_from_growth_rate # R_CrI ile karşılaştır

Yukarıdaki tahminlerin dallanma süreci modeli kullanılarak elde edilenlerden biraz farklı olduğuna dikkat edin. 
Bunun birkaç nedeni vardır. 
İlk olarak, dallanma süreci (EpiEstim) tahmini için daha ayrıntılı veriler (haftalık vs günlük insidans) kullandınız. 
Ayrıca, log-lineer model tüm veri noktalarına aynı ağırlığı verirken, dallanma süreci modeli her veri noktasına farklı ağırlık verir (her zaman adımında gözlemlenen insidansa bağlı olarak). 
Bu, biraz farklı R tahminlerine yol açabilir. 

## Kısa vadeli insidans öngörüsü  

`projections` paketindeki `project` fonksiyonu, `EpiEstim`'de `$R$`'yi tahmin etmek için kullanılan aynı dallanma sürecini kullanarak günlük insidansı simüle ederek makul salgın yörüngelerini simüle etmek için kullanılabilir. Tek ihtiyaç duyulan bir veya birkaç `$R$` değeri ve `distcrete` nesnesi olarak saklanan bir seri aralık dağılımıdır.  

Burada, önce yukarıdan R'nin medyan tahmini olan sabit bir `$R$` = `r signif(R_median, 3)` değerini kullanarak 5 rastgele yörüngeyi nasıl simüle edebileceğimizi gösteriyoruz.  
Gelecekteki insidansı simüle etmek için yukarıdaki aynı günlük kesilmiş insidans nesnesini kullanın.  

In [ ]:
#?project
small_proj <- project(i_daily_trunc,# insidans nesnesi 
                      R = R_median, # kullanılacak R tahmini
                      si = si,      # seri aralık dağılımı
                      n_sim = 5,    # 5 yörünge simüle et
                      n_days = 10,  # 10 gün boyunca
                      R_fix_within = TRUE) # her gün aynı R değerini koru

# her öngörülen yörüngeye bak (sütunlar olarak):
as.matrix(small_proj)

Aynı prensibi kullanarak, makul `$R$` değerleri aralığını kullanarak önümüzdeki 2 hafta için 1.000 yörünge oluşturun.  
R'nin posterior dağılımı gamma dağılımlıdır (Cori ve ark. AJE 2013'e bakın) bu nedenle bu dağılımdan rastgele değerler çekmek için `rgamma` fonksiyonunu kullanabilirsiniz. Aşağıda gösterildiği gibi `epitrix` paketindeki `gamma_mucv2shapescale` fonksiyonunu da kullanmanız gerekecektir.  

In [ ]:

sample_R <- function(R, n_sim = 1000)
{
  mu <- R$R$`Mean(R)`
  sigma <- R$R$`Std(R)`
  Rshapescale <- gamma_mucv2shapescale(mu = mu, cv = sigma / mu)
  R_sample <- rgamma(n_sim, shape = Rshapescale$shape, scale = Rshapescale$scale)
  return(R_sample)
}

R_sample <- sample_R(R, 1000) # posterior dağılımdan 1000 R değeri örnekle
hist(R_sample, col = "grey")  # örneğin histogramını çiz
abline(v = R_median, col = "red") # tahmin edilen medyan R'yi kırmızı düz dikey çizgi olarak göster
abline(v = R_CrI, col = "red", lty = 2) # R'nin %95CrI'sını kırmızı kesikli dikey çizgiler olarak göster

Yeni projeksiyonlarınızın sonuçlarını `proj` adlı bir nesnede saklayın.

In [ ]:
proj <- project(x = i_daily_trunc, 
                R = R_sample, # şimdi R değerlerinin bir örneğini kullanıyor
                si = si, 
                n_sim = 1000, 
                n_days = 14, # 2 hafta boyunca öngör
                R_fix_within = TRUE)


Projeksiyonları şu şekilde görselleştirebilirsiniz:  

In [ ]:
plot(i_daily_trunc) %>% add_projections(proj, c(0.025, 0.5, 0.975))

Projeksiyonların aşağıdaki özetini nasıl yorumlarsınız?  

In [ ]:
apply(proj, 1, summary)
apply(proj, 1, function(x) mean(x > 0)) # her belirtilen günde en az 
                                        # bir vakası olan yörüngelerin oranı

apply(proj, 1, mean) # günlük ortalama vaka sayısı 

apply(apply(proj, 2, cumsum), 1, summary) # önümüzdeki iki haftada öngörülen 
                                          # kümülatif vaka sayısı

Bu sonuçlara göre, yakın gelecekte daha fazla vakanın ortaya çıkma şansı nedir? Bu salgın kontrol altına alınıyor mu? Müdahaleyi büyütmeyi / küçültmeyi önerir misiniz? Bu, $$R$$ tahmininizle tutarlı mı?
 

## Dur!

Daha ileri gitmeden önce bu noktaya ulaştığınızda lütfen bir gösterici kişiye haber verin. 

## Üreme sayısını tahmin ederken seri aralık tahminlerindeki belirsizliği hesaba katma

Bu bölümün aşağıdaki bölümlerden bağımsız olduğunu unutmayın, zamanınız yoksa atlayın. 

EpiEstim, bulaştıran/bulaşan birey çiftlerinin sınırlı örnek boyutu nedeniyle seri aralık tahminlerindeki belirsizliği açıkça hesaba katmaya izin verir. Ayrıca bu çiftlerin her biri için semptom başlangıcı tarihlerindeki belirsizliği hesaba katmaya da izin verdiğini unutmayın (burada gerekli değildir). 

`estimate_R`'de `method = "si_from_data"` seçeneğini kullanın. 
Bu seçeneği kullanmak için 4 sütunlu bir veri çerçevesi oluşturmanız gerekir: 
`EL`, `ER`, `SL` ve `SR` bulaştıran (E) ve bulaşan (S ikincil için) vakalarda gözlemlenen semptom zamanının sol (L) ve sağ (R) sınırları için. Burada bunu `si_obs`'tan şu şekilde türetiyoruz:

In [ ]:
si_data <- data.frame(EL = rep(0L, length(si_obs)), 
                      ER = rep(1L, length(si_obs)), 
                      SL = si_obs, SR = si_obs + 1L)

Daha sonra bunu `estimate_R`'ye besleyebiliriz (ancak MCMC kullanarak seri aralık dağılımını tahmin ettiği ve üreme sayısını tahmin etmek için seri aralıktaki belirsizliği tam olarak hesaba kattığı için çalışması biraz zaman alacaktır).

In [ ]:
config <- make_config(t_start = 2, 
                      t_end = length(i_daily_trunc$counts))
R_variableSI <- estimate_R(i_daily_trunc, method = "si_from_data", 
                           si_data = si_data,
                           config = config)
# MCMC'nin yakınsadığını kontrol etme: 
R_variableSI$MCMC_converged
# ve çıktıyı çizme:
plot(R_variableSI)

Yeni medyan tahmin edilen R ve %95CrI'ye bakın: önceki tahminlerinizden ne kadar farklı? Temaslar veri setinin boyutunun sonuçlarınız üzerinde etkisi olduğunu düşünüyor musunuz? 

In [ ]:
R_variableSI_median <- R_variableSI$R$`Median(R)`
R_variableSI_CrI <- c(R_variableSI$R$`Quantile.0.025(R)`, R_variableSI$R$`Quantile.0.975(R)`)
R_variableSI_median
R_variableSI_CrI

## Zamana göre değişen bulaştırıcılığı tahmin etme  

($$R$$)'nin zaman içinde sabit olduğu varsayımı sürdürülemez hale geldiğinde, bir alternatif anlık üreme sayısını ($$R_t$$) kullanarak zamana göre değişen bulaştırıcılığı tahmin etmektir. Cori ve ark. (2013) tarafından tanıtılan bu yaklaşım, `EpiEstim` paketinde de uygulanmıştır. Yukarıda açıklanan aynı Poisson olabilirliği kullanarak özel zaman pencereleri için (varsayılan, ardışık kayan zaman pencerelerinin bir dizisidir) ($$R_t$$) tahmin eder. Aşağıda, 1 haftalık kayan zaman pencereleri için bulaştırıcılığı tahmin ediyoruz (`estimate_R`'nin varsayılanı):  

In [ ]:
config = make_config(list(mean_si = si_fit$mu, std_si = si_fit$sd))  
# t_start ve t_end varsayılan olarak 1 haftalık kayan pencereler üzerinde R tahmin etmek için otomatik olarak ayarlanır. 

In [ ]:
Rt <- estimate_R(incid = i_daily_trunc,      # insidans nesnesi
                 method = "parametric_si",   # parametrik seri aralık kullan
                 config = config)            # yukarıda belirtilen yapılandırma

# en son Rt tahminlerine bak:
tail(Rt$R[, c("t_start", "t_end", "Median(R)", 
             "Quantile.0.025(R)", "Quantile.0.975(R)")])

Zaman içinde `$R$` tahminini çizin:

In [ ]:
plot(Rt, legend = FALSE)

Bu sonucu nasıl yorumlarsınız? Bu gösterimin uyarısı nedir?

Yukarıdaki gibi `i_daily_trunc` kullanmak yerine tüm salgın eğrisini yani `i_daily`'yi kullanmış olsaydınız ne sonuca varırdınız? 

In [ ]:
Rt_whole_incid <- estimate_R(incid = i_daily, 
                             method = "parametric_si", 
                             config = config) 

tail(Rt_whole_incid$R[, c("t_start", "t_end", 
                         "Median(R)", "Quantile.0.025(R)", "Quantile.0.975(R)")])  
# yukarıdaki yanlış olarak son bulaştırıcılığın <1 olduğunu çıkarır

In [ ]:
cat("- yukarıdaki kayan zaman penceresi içinde sabit R varsayar")  
